# Feature Engineering
###  The file contains steps to form various technical indicators

In [1]:
import talib
import pandas as pd
import numpy as np

## Getting Data - Consumer Discretionary Sector
 Since our sample is very large, we choose to focus on Consumer Discretionary sector

In [38]:
Discretionary = 'Consumer Discretionary'
df = pd.read_csv(r'Russell_universe.csv')
df = df[df['Sector'] == Discretionary]

In [40]:
companies=list(df['Ticker'])
companies

['SPWH UW Equity',
 'HD UN Equity',
 'MCD UN Equity',
 'AZO UN Equity',
 'BBY UN Equity',
 'HRB UN Equity',
 'CCL UN Equity',
 'BIG UN Equity',
 'TGT UN Equity',
 'M UN Equity',
 'F UN Equity',
 'PRPL UR Equity',
 'GPS UN Equity',
 'GPC UN Equity',
 'HOG UN Equity',
 'KSS UN Equity',
 'LEG UN Equity',
 'LEN UN Equity',
 'MCFT UQ Equity',
 'LB UN Equity',
 'LOW UN Equity',
 'MAT UW Equity',
 'NKE UN Equity',
 'JCP UN Equity',
 'PHM UN Equity',
 'TIF UN Equity',
 'TJX UN Equity',
 'VNCE UN Equity',
 'VFC UN Equity',
 'GHC UN Equity',
 'WHR UN Equity',
 'JWN UN Equity',
 'MOD UN Equity',
 'BBBY UW Equity',
 'DHI UN Equity',
 'ATGE UN Equity',
 'SEAS UN Equity',
 'HAS UW Equity',
 'ROST UW Equity',
 'AN UN Equity',
 'SBUX UW Equity',
 'ORLY UW Equity',
 'BWA UN Equity',
 'URBN UW Equity',
 'HUD UN Equity',
 'DLTR UW Equity',
 'DRI UN Equity',
 'ANF UN Equity',
 'AMZN UW Equity',
 'RL UN Equity',
 'YUM UN Equity',
 'EBAY UW Equity',
 'BKNG UW Equity',
 'TPR UN Equity',
 'SIC UR Equity',
 'G

<h2> Simple Rules

### SMA Rules：
1. **Trend Analysis**
<br> The basic rule for trading with the SMA is that a security trading above its SMA is in an uptrend, while a security trading below its SMA is in a downtrend.
2. **Golden/Death Cross**
<br>A death cross occurs when the 50-day simple moving average crosses below the 200-day moving average. This is considered a bearish signal, that further losses are in store. 
<br>The golden cross occurs when a short-term moving average breaks above a long-term moving average. Reinforced by high trading volumes, this can signal further gains are in store.

In [5]:
def crossover(shortterm, longterm):
    if len(shortterm)==len(longterm):
        temp = np.zeros(len(shortterm))
        for i in range(1, len(shortterm)):
            temp[i] = 1 if shortterm[i] >= longterm[i] and shortterm[i-1] < longterm[i-1] \
                                   else -1 if shortterm[i] <= longterm[i] and shortterm[i-1] > longterm[i-1] \
                                   else 0
    else:
        raise ValueError('Lengths do not match')
    return temp

### WMA Rules：
* Use the same rules that apply to SMA when interpreting WMA

### BB Band Rules：
1. To buy when price is near to the lower band and sell then price is near to the upper band.(When the band is parallel)
2. Avoid buying in an downtrend and selling in a uptrend.
3. **<font color=#DC143C>Measeure the volability by the wide of range. A narrow range guarantees the movements are going either up or down.**

In [6]:
def bollinger_band(close, nbdevup=3, nbdevdn=3,timeperiod=20):
    upperBB20, middleBB20, lowerBB20 = talib.BBANDS(close, nbdevup, nbdevdn, timeperiod)
    bbp = (close - lowerBB20) / (upperBB20 - lowerBB20)
    temp = np.array([1 if x < 0 else -1 if x>1 else 0 for x in bbp])
    return temp

In [7]:
def MA_trend(close, MA_days):
     #Moving averages, for example, can be used to identify the overall trend. 
     #If the price is lower than a moving average, the stock is likely to be in a downtrend, and vice versa for an uptrend.
    close_MA = talib.EMA(close, timeperiod=MA_days)
    trend = crossover(close, close_MA)
    return trend

def DX_trend(high, low, close, ADX_days = 25, DMI_days = 14):
    #When the +DMI is above the -DMI, prices are moving up 
    #When the -DMI is above the +DMI, prices are moving down, and ADX measures the strength of the downtrend. 
    #Many traders will use ADX readings above 25 to suggest that the trend is strong enough for trend-trading strategies. 
    #Conversely, when ADX is below 25, many will avoid trend-trading strategies.
    ADX = talib.ADX(high, close, low, timeperiod = ADX_days)
    PMI = talib.PLUS_DI(high, low, close, timeperiod = DMI_days)
    NMI = talib.MINUS_DI(high, low, close, timeperiod = DMI_days)
    ADX_trend = ADX > 25        # the trend is strong
    PMI_trend = PMI > NMI       # +DMI is above the -DMI, prices are moving up,
    NMI_trend = PMI < NMI       # -DMI is above the +DMI, prices are moving down
    trend = np.where(PMI_trend & ADX_trend, 1, np.where(NMI_trend & ADX_trend > 25, -1, 0)) 
    return trend

4. BB squeeze: After a tight **consolidation**, the prices are tempt to make a larger move in either direction, ideally in higher volume. Expanding a volume on a breakout is a sign that traders are expect that price will continue to move in the breakout direction. The **longer** it moves in the narrow band, the more likely it evetually to **penatrate these band and eventually continue on in the direction of breakout**, if this event occurs in the direction of previous established longer-term trend. It significants that a potential explosion in price action is about to accur, in the direction of the candlestick pushing againast the band. The **more vertical, the stronger the potential move.**

In [8]:
def KELCH(high, low, close, Kel_n):  
    KelChM = ((high + low + close) / 3).rolling(Kel_n).mean()
    KelChU = ((4 * high - 2 * low + close) / 3).rolling(Kel_n).mean()
    KelChD = ((-2 * high + 4 * low + close) / 3).rolling(Kel_n).mean()
    return KelChU, KelChM, KelChD

In [9]:
# BB squeeze
def BB_squeeze(low, high, close, timeperiod, nbdevup=2, nbdevdn=2):
    BBU, BBM, BBL = talib.BBANDS(close, timeperiod, nbdevup=2, nbdevdn=2, matype=0)
    KelChU, KelChM, KelChD = KELCH(high, low, close, timeperiod)
    squeeze = 0
    waiting_up = 0
    waiting_down = 0
    signal = np.zeros(len(high))
    for i in range(len(high)):
        if BBU[i] < KelChU[i] and BBL[i] > KelChD[i]:    ###### find a squeeze
            squeeze = 1
            waiting_up = 0
            waiting_down = 0
        if squeeze == 1:
            if BBU[i] > KelChU[i] or BBL[i] < KelChD[i]: 
                squeeze = 0
                #Buy: When a squeeze is formed, wait for the upper Bollinger Band® to cross upward through the upper Keltner Channel, and then wait for the price to break the upper band for a long entry.
                #Sell: When a squeeze is formed, wait for the lower Bollinger Band® to cross through the downward lower Keltner Channel, and then wait for the the price to break the lower band for a short entry.
                if BBU[i] > KelChU[i]:
                    waiting_up = 1
                if BBL[i] < KelChD[i]:
                    waiting_down = 1
        if waiting_up == 1 and close[i] > BBU[i] and close[i-1] < BBU[i-1]:
            signal[i] = 1
            waiting_up = 0
        if waiting_down == 1 and close[i] < BBL[i] and close[i-1] > BBL[i-1]:
            signal[i] = -1
            waiting_down = 0
    return signal

### EMA Rules：
* Use the same rules that apply to SMA when interpreting WMA

### DEMA Rules：
1. Use the same rules that apply to SMA when interpreting WMA
2. **<font color=#DC143C>DEMA can be used in a variety of indicators in which the logic is based on a moving average.(e.g. MACD, TRIX) (WHEN?)</font>**

### KAMA Rules：
 real = KAMA(close, timeperiod=30)
1. A cross above or below KAMA indicates directional changes in prices
2. Combine signals and techniques. Chartists can use a longer-term KAMA to define the bigger trend and a shorter-term KAMA for trading signals. 
<br>AND [KAMA(10,5,30) >  Daily SMA(50,KAMA(10,5,30))] AND [Daily Close crosses KAMA(10,2,30)] 

In [10]:
def KAMA_combine(close, KAMA_n=30, SMA_n=50):
    KAMA = talib.KAMA(close, timeperiod = KAMA_n)
    SMA_KAMA = talib.SMA(KAMA, timeperiod = SMA_n)
    
    # Chartists can use a longer-term KAMA to define the bigger trend and a shorter-term KAMA for trading signals.
    up_trend = KAMA > SMA_KAMA
    down_trend = KAMA < SMA_KAMA
    
    # A cross above or below KAMA indicates directional changes in prices
    signal = np.zeros(len(close))
    up_cross = np.array([True if x==1 else False for x in crossover(close, KAMA)])
    down_cross = np.array([True if x==-1 else False for x in crossover(close, KAMA)])
    
    signal = np.where(up_cross & up_trend, 1, 
                      np.where(down_cross & down_trend, -1, 0))
    return signal

### MAVP Rules：
 real = MAVP(close, periods, minperiod=2, maxperiod=30, matype=0)
* **<font color=#DC143C>Require periods, which we don't know.</font>**

### MIDPOINT Rules:
real = MIDPOINT(close, timeperiod=14)

### MIDPRICE Rules：
real = MIDPRICE(high, low, timeperiod=14)

### SAR Rules：
real = SAR(high, low, acceleration=0, maximum=0)
1. The basic use of the Parabolic SAR is to buy when the dots move below the price bars (signaling an uptrend) and sell/short-sell when the dots move above the price bars (signaling a downtrend).
2. It is better to analyze the price action of the day to determine whether the trend (if there is one) is up or down. Another indicator, such as a moving average or trendlines, can also be used to establish the overall trend direction. If there is a trend, only take trade signals in the direction of the overall trend.

In [11]:
def sar(high, low, close, ADX_days = 25, DMI_days = 14):
    trend = DX_trend(high, low, close, ADX_days, DMI_days)
    sar = talib.SAR(high, low, acceleration=0, maximum=0)
    sar_signal = crossover(close, sar)
    signal =  np.array([1 if i==2 else -1 if i==-2 else 0 for i in [z+y for (z,y) in zip(sar_signal, trend)]])
    return signal

### TEMA Rules：
real = TEMA(close, timeperiod=30)
1. TEMA only works on a strong trend.So we can use ADX to help to decide.

In [12]:
def TEMA(high, low, close, TEMA_days = 30, ADX_days = 25, DMI_days = 14):
    trend = DX_trend(high, low, close, ADX_days, DMI_days)
    TEMA = talib.TEMA(close, timeperiod = TEMA_days)
    TEMA_signal = crossover(close, TEMA)
    signal =  np.array([1 if i==2 else -1 if i==-2 else 0 for i in [z+y for (z,y) in zip(TEMA_signal, trend)]])
    return signal

### AD line rules:
real = AD(high, low, close, volume)
1. If the slope of the A/D line is up and the market is trending upward, then the market is said to be healthy.Conversely, if the indexes are continuing to move lower and the A/D line has turned upwards, called bullish divergence, it may be an indication that the sellers are losing their conviction. If the A/D line and the markets are both trending lower together, there is a greater chance that declining prices will continue.

In [13]:
def ADline(high, low, close, volume, ADX_days = 25, DMI_days = 14, MA_days = 50):
    trend = DX_trend(high, low, close, ADX_days, DMI_days)  # this is the trend of stock price movement
    
    AD = talib.AD(high, low, close, volume)
    AD_trend = MA_trend(AD, MA_days)   # this is trend of OBV movement    
    
    signal =  np.array([1 if i==2 else -1 if i==-2 else 0 for i in [z+y for (z,y) in zip(AD_trend, trend*(-1))]])
    return signal

### OBV rules:
real = OBV(close, volume)
1. If the price is going up and the volumn is going down, people are losing confidence in this stock. This is a signal of price decline.
2. If the price is going down and the volumns is going up, there will be a reverse in price, which is buy pressure.

In [14]:
def OBV(high, low, close, volume, ADX_days = 25, DMI_days = 14, MA_days = 50):
    trend = DX_trend(high, low, close, ADX_days, DMI_days)  # this is the trend of stock price movement
    
    OBV = talib.OBV(close, volume) 
    OBV_trend = MA_trend(OBV, MA_days)   # this is trend of OBV movement    
    
    signal =  np.array([1 if i==2 else -1 if i==-2 else 0 for i in [z+y for (z,y) in zip(OBV_trend, trend*(-1))]])
    return signal

### ATR rules:
real = ATR(high, low, close, timeperiod=14)
1. **Exit Sign**: whenever price closes more than one ATR below the most recent close, a significant change in the nature of the market has occurred. Closing a long position becomes a safe bet, because the stock is likely to enter a trading range or reverse direction at this point.
2. Whenever price closes more than an ATR above the most recent close, a change in volatility has occurred. Taking a long position is betting that the stock will follow through in the upward direction.

In [15]:
def ATR(high, low, close, ATR_days=14, ADX_days = 25, DMI_days = 14):
    trend = DX_trend(high, low, close, ADX_days, DMI_days)  # this is the trend of stock price movement
    
    ATR =  talib.ATR(high, low, close, timeperiod = ATR_days)

    signal = np.zeros(len(high))
    for i in range(1, len(close)):
        if close[i]-close[i-1] >= ATR[i]:
            if trend[i] == 1:
                signal[i] = 1
            elif trend[i] == -1:
                signal[i] = -1
    return signal

### Hilbert Transform(HT) Rules
inphase, quadrature = HT_PHASOR(close)
* BUY - Place Buy position when Blue (InPhase) line crosses above Red (Quadrature) line below the zero level.
* SELL - Place Sell position when Blue (InPhase) line crosses below Red (Quadrature) line above the zero level.

In [16]:
def HT(close):
    inphase, quadrature = talib.HT_PHASOR(close)
    signal = crossover(inphase, quadrature)
    return signal

### Chande Momentum Oscillator (CMO) Rules
real = CMO(close, timeperiod=14)
* The oscillator generates a bullish signal when it crosses above the moving average and a bearish signal when it drops below the moving average.
* The oscillator can be used as a confirmation signal when it crosses above or below the 0 line. For example, if the 50-day moving average crosses above the 200-day moving average (golden cross), a buy signal is confirmed when the Chande momentum oscillator crosses above 0, predicting prices are headed higher. 

In [17]:
def cmo(close, cmo_period, ma_period):
    res = np.zeros(len(close))
    CMO = np.array(talib.CMO(close, timeperiod=cmo_period))
    MA = np.array(talib.MA(close, timeperiod = ma_period))
    
    res = crossover(CMO,MA)
    
    return res

### MACD Rules
1. MACD crossing above zero is considered bullish, while crossing below zero is bearish. Secondly, when MACD turns up from below zero it is considered bullish. When it turns down from above zero it is considered bearish.
2. When the MACD line crosses from below to above the signal line, the indicator is considered bullish. The further below the zero line the stronger the signal.
3. When the MACD line crosses from above to below the signal line, the indicator is considered bearish. The further above the zero line the stronger the signal.

In [18]:
def macd(close,fastp,slowp,signalp):
    macd, macd_signal, macdhist = np.array(talib.MACD(close, fastperiod=fastp, slowperiod=slowp, signalperiod=signalp))
    
    res = crossover(macd,macd_signal)

    return res

### ADX rules : 
The traditional setting for the ADX indicator is 14 time periods, but analysts have commonly used the ADX with settings as low as 7 or as high as 30. Lower settings will make the average directional index respond more quickly to price movement but tend to generate more false signals. Higher settings will minimize false signals but make the average directional index a more lagging indicator.

### APO ( Absolute Price Oscillator):
1. APO crossing above zero is considered bullish, while crossing below zero is bearish.
2. A positive indicator value indicates an upward movement, while negative readings signal a downward trend.

In [20]:
def apo(close,fastp,slowp):
    apo_ = np.array(talib.APO(close, fastperiod=fastp, slowperiod=slowp, matype=0))
    res = np.zeros(len(close))
    for i in range(1,len(apo_)):
        if (apo_[i-1] <= 0) and (apo_[i] >0):
            res[i] = 1                            #buy
        elif (apo_[i-1] >= 0) and (apo_[i] <0):
            res[i] = -1                          #sell
            
    return res

## AROON
When the oscillator moves above the zero line it means the Aroon Up is crossing above the Aroon Down. This means the price has made a high more recently than a low. That could be a sign that an uptrend is starting.

https://www.investopedia.com/terms/a/aroonoscillator.asp

In [21]:
def aroon (high,low,timep=14):
    aroondown,aroonup = np.array(talib.AROON(df['High'], df['Low'], timeperiod=timep))
    aroonosc = np.zeros(len(high))
    aroonosc = aroonup - aroondown    # aroon oscillator
                                 
    res = np.zeros(len(high))
    for i in range(1,len(aroonosc)):
        if aroonosc[i-1] <= 0 and aroonosc[i] >0:
            res[i] = 1
        elif aroonosc[i-1] >=0 and aroonosc[i] <0:
            res[i] = -1 
                                 
    return res

## ROC
Positive values indicate upward buying pressure or momentum, while negative values below zero indicate selling pressure or downward momentum.


In [22]:
def roc(close, timep= 10):
    roc_ =np.array( talib.ROC(close, timeperiod=timep))
    res = np.zeros(len(close))
    
    for i in range(1,len(close)):
        if (roc_[i-1] <= 0) and (roc_[i] >0):
            res[i] = 1
        elif (roc_[i-1] >= 0) and (roc_[i] <0):
            res[i] = -1 
                                 
    return res

<h2> Merge Simple Rules of All Companies

In [ ]:
all_df = list()
null_companies = list()
for company in companies:
    try:
        # Get the data from one company
        company = company.replace('/',' ')
        file='./companies/'+company+'.csv'
        df=pd.read_csv(file, header=5).drop(0)
        df = df.dropna(how='any')   # the dates might not be consecutive
        df.reset_index(drop=True, inplace=True)
        df['Dates']=pd.to_datetime(df['Dates'])

        ########### Simple rules ##################
        #------------SMA rules-------------------
        df['SMA14'] = crossover(df['PX_LAST'], talib.SMA(df['PX_LAST'], timeperiod=15))
        df['SMA50'] = crossover(df['PX_LAST'], talib.SMA(df['PX_LAST'], timeperiod=50))
        df['SMA200'] = crossover(df['PX_LAST'], talib.SMA(df['PX_LAST'], timeperiod=200))

        df['SMA5-10'] = crossover(talib.SMA(df['PX_LAST'], timeperiod=5), talib.SMA(df['PX_LAST'], timeperiod=10))
        df['SMA50-200'] = crossover(talib.SMA(df['PX_LAST'], timeperiod=50), talib.SMA(df['PX_LAST'], timeperiod=200))

        #------------WMA rules-------------------
        df['WMA14'] = crossover(df['PX_LAST'], talib.WMA(df['PX_LAST'], timeperiod=15))
        df['WMA50'] = crossover(df['PX_LAST'], talib.WMA(df['PX_LAST'], timeperiod=50))
        df['WMA200'] = crossover(df['PX_LAST'], talib.WMA(df['PX_LAST'], timeperiod=200))

        df['WMA5-10'] = crossover(talib.WMA(df['PX_LAST'], timeperiod=5), talib.WMA(df['PX_LAST'], timeperiod=10))
        df['WMA50-200'] = crossover(talib.WMA(df['PX_LAST'], timeperiod=50), talib.WMA(df['PX_LAST'], timeperiod=200))

        #------------WMA rules-------------------
        df['WMA14'] = crossover(df['PX_LAST'], talib.WMA(df['PX_LAST'], timeperiod=15))
        df['WMA50'] = crossover(df['PX_LAST'], talib.WMA(df['PX_LAST'], timeperiod=50))
        df['WMA200'] = crossover(df['PX_LAST'], talib.WMA(df['PX_LAST'], timeperiod=200))

        df['WMA5-10'] = crossover(talib.WMA(df['PX_LAST'], timeperiod=5), talib.WMA(df['PX_LAST'], timeperiod=10))
        df['WMA50-200'] = crossover(talib.WMA(df['PX_LAST'], timeperiod=50), talib.WMA(df['PX_LAST'], timeperiod=200))

        #------------EMA rules-------------------
        df['EMA14'] = crossover(df['PX_LAST'], talib.EMA(df['PX_LAST'], timeperiod=15))
        df['EMA50'] = crossover(df['PX_LAST'], talib.EMA(df['PX_LAST'], timeperiod=50))
        df['EMA200'] = crossover(df['PX_LAST'], talib.EMA(df['PX_LAST'], timeperiod=200))

        df['EMA5-10'] = crossover(talib.EMA(df['PX_LAST'], timeperiod=5), talib.EMA(df['PX_LAST'], timeperiod=10))
        df['EMA50-200'] = crossover(talib.EMA(df['PX_LAST'], timeperiod=50), talib.EMA(df['PX_LAST'], timeperiod=200))

        #------------DEMA rules---------------------
        df['DEMA14'] = crossover(df['PX_LAST'], talib.DEMA(df['PX_LAST'], timeperiod=15))
        df['DEMA50'] = crossover(df['PX_LAST'], talib.DEMA(df['PX_LAST'], timeperiod=50))
        df['DEMA200'] = crossover(df['PX_LAST'], talib.DEMA(df['PX_LAST'], timeperiod=200))

        df['DEMA5-10'] = crossover(talib.DEMA(df['PX_LAST'], timeperiod=5), talib.DEMA(df['PX_LAST'], timeperiod=10))
        df['DEMA50-200'] = crossover(talib.DEMA(df['PX_LAST'], timeperiod=50), talib.DEMA(df['PX_LAST'], timeperiod=200))    

        #-----------BB band rules---------------------
        df['BB_crossover'] = bollinger_band(df['PX_LAST'], nbdevup=3, nbdevdn=3,timeperiod=20)
        df['BB_squeeze'] = BB_squeeze(df['PX_LOW'], df['PX_HIGH'], df['PX_LAST'], timeperiod=20, nbdevup=2, nbdevdn=2)

        #------------KAMA rules------------------------
        df['KAMA_crossover'] = KAMA_combine(df['PX_LAST'], KAMA_n=30, SMA_n=50)

        #----------- SAR rules -------------------------
        df['SAR_crossover'] = sar(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], ADX_days = 25, DMI_days = 14)

        #------------ TEMA rules ------------------------
        df['TEMA_crossover'] = TEMA(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], TEMA_days = 30, ADX_days = 25, DMI_days = 14)

        #------------ AD line rules ---------------------
        df['OBV_divergence'] = OBV(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], df['PX_VOLUME'], ADX_days = 25, DMI_days = 14, MA_days = 50)

        #------------ ATR rules --------------------------
        df['ATR_close'] = ATR(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], ATR_days=14, ADX_days = 25, DMI_days = 14)

        #------------ HT Rules ---------------------------
        df['HT_crossover'] = HT(df['PX_LAST'])
        
        #------------ CMO Rules --------------------------
        df['CMO_crossover'] = cmo(df['PX_LAST'], cmo_period = 25, ma_period = 50)
        
        #------------ MACD Rules --------------------------
        df['MACD_12_29_9'], df['MACD_signal_12_29_9'], df['MACD_hist_12_29_9'] = macd(df['PX_LAST'], fastp = 12, slowp = 29,signalp = 9)
        df['MACD_5_35_5'], df['MACD_signal_5_35_5'], df['MACD_hist_5_35_5'] = macd(df['PX_LAST'], fastp = 5, slowp = 35,signalp = 5)
        
        #------------ APO Rules --------------------------
        df['apo_'] = talib.APO(df['PX_LAST'], fastperiod=14, slowperiod=30, matype=0)
        df['apo_'] = talib.APO(df['PX_LAST'], fastperiod=16, slowperiod=35, matype=0)
        
        #------------ AROON Rules --------------------------
        df['aroondown'],df['aroonup']  = talib.AROON(df['PX_HIGH'], df['PX_LOW'], timeperiod=14)
        
        #------------ ROC Rules --------------------------
        df['ROC'] = roc(df['PX_LAST'], timeperiod=10)
        
        #------------ STOCH Rules --------------------------
        df['slowk'], df['slowd'] = talib.STOCH(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        df['slowk'], df['slowd'] = talib.STOCH(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], fastk_period=10, slowk_period=6, slowk_matype=0, slowd_period=5, slowd_matype=0)
        df['slowk'], df['slowd'] = talib.STOCH(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], fastk_period=15, slowk_period=9, slowk_matype=0, slowd_period=7, slowd_matype=0)
        #-----------------------Return Rate-----------------------------
        periods = [3,5,10]   # unit = day
        thresholds = [0.03, 0.05, 0.10]
        for i in periods:
            period = i
            df['return'+str(period)] = df['PX_LAST'].pct_change(periods = period)
        for i in range(len(periods)):
            df['signal'+str(periods[i])] = np.where(df['return'+str(periods[i])] > thresholds[i], 
                                                    1 ,
                                                   np.where(df['return'+str(periods[i])] < -thresholds[i],-1,0)
                                                   )
        #Output all indicators of this company
#         df=df.dropna(how='any')
#         df.to_csv('C:/Users/Administrator/Documents/balyansy/indicators/indicator_'+company+'.csv')

        # concanate the data from all companies
        all_df.append(df)
    except:
        null_companies.append(company)
    
data_set = pd.concat(all_df)
data_set.to_csv(Discretionary +'_all_companies.csv')